In [1]:
%load_ext autoreload
%autoreload 2

# Environment prep for Colab
 - download git repo 
 - install required pip packages

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')
#!mkdir /content/drive/My\ Drive/Results
#MAIN_SAVE_DIR = '/content/drive/My Drive/Results'
#IMG_DIR = '/content/Generative_Models/data/.pokemon' #change .qbert here as well!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
#%cd /content/Generative_Models/
#!pip -q install -r /content/Generative_Models/requirements.txt
#!pip install colorlog==4.0.2
#%cd /content/

# Connect Drive to save and restore results

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')
#!mkdir /content/drive/My\ Drive/Results
#MAIN_SAVE_DIR = '/content/drive/My Drive/Results'
#IMG_DIR = '/content/Generative_Models/data/.pokemon' #change .qbert here as well

# Settings

In [4]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [5]:
!pip -q install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [6]:
import tensorflow as tf
print(tf.__version__)

2.3.1


In [7]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

C:\Users\Kriple\PycharmProjects\Generative_Models


# only on colab

In [8]:
#!cat data/.pokemon/pokemon.tar.part* > data/.pokemon/pokemon.tar

In [9]:
#!tar -xvf data/.pokemon/pokemon_combined.tar --directory data/.pokemon/

# Dataset loading

In [10]:
from training.generators.from_lmdb.lmdb_image_generator import get_generators

In [11]:
dataset_name='pokemon'
inputs_shape= image_shape=(100, 100, 3)
batch_size = 10
latents_dim = 100
intermediate_dim = 50
lmdb_dir = 'data/.pokemon_LMDB'

In [12]:
from transformation.lmdb_transformer import LmdbTransformer

In [13]:
train_generator, val_generator = get_generators(
        lmdb_dir=lmdb_dir,
        batch_size=batch_size,
        episode_len=None,
        episode_shift=None
)

Initializing Iterator training Number of images 591
Initializing Iterator validation Number of images 218


C:\Users\Kriple\PycharmProjects\Generative_Models\transformation\file_image_generator.py:51: UserWarning: No files found
  warnings.warn(msg)


In [14]:
import numpy as np
_instance_scale=1.0
for data in val_generator:
    print(np.max(data['images']))
    break
    #print(data.keys())

1.0


In [15]:
data['images'][0].shape

TensorShape([100, 100, 3])

In [16]:
data.keys()

dict_keys(['images', 'label'])

# Model's Layers definition

In [17]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    flat_outputs_shape = np.prod(inputs_shape)
flat_outputs_shape

30000

In [18]:
latents_dim

100

In [19]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),

            tf.keras.layers.Dense(units=flat_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [20]:
model_name = dataset_name+'AE_Dense_reconst_ell'
experiments_dir='experiments'+sep_local+model_name

In [21]:
from training.autoencoding_basic.autoencoders.autoencoder import autoencoder as AE

In [22]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [23]:
from utils.data_and_files.file_utils import create_if_not_exist

In [24]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [25]:
create_if_not_exist(_restore)
_restore

'experiments\\pokemonAE_Dense_reconst_ell\\var_save_dir'

In [26]:
#to restore trained model, set filepath=_restore

In [27]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=_restore
    )

  DEBUG    | Restore old models ...
  DEBUG    | Restore old models ...
  ERROR    | Faild to restore old models !
  ERROR    | Faild to restore old models !



SavedModel file does not exist at: experiments\pokemonAE_Dense_reconst_ell\var_save_dir\pokemonAE_Dense_reconst_ell_inference.hdf5/{saved_model.pbtxt|saved_model.pb}




  WARNING  | None
  WARNING  | None


Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 100, 100, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 100, 100, 50)      200       
_________________________________________________________________
dense_1 (Dense)              (None, 100, 100, 50)      2550      
_________________________________________________________________
flatten (Flatten)            (None, 500000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               50000100  
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0 

  WARNING  | None
  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 100)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 1666)              168266    
_________________________________________________________________
dense_5 (Dense)              (None, 1666)              2777222   
_________________________________________________________________
dense_6 (Dense)              (None, 1666)              2777222   
_________________________________________________________________
dense_7 (Dense)              (None, 30000)             50010000  
_________________________________________________________________
reshape (Reshape)            (None, 100, 100, 3)       0

In [28]:
ae.compile(metrics=None)

Model: "pokemonAE_Dense_reconst_ell"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 100, 100, 3)]     0         
_________________________________________________________________
inference (Functional)       (None, 100)               50003250  
_________________________________________________________________
generative (Functional)      (None, 100, 100, 3)       55742822  
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 100, 100, 3)]     0         
Total params: 105,746,072
Trainable params: 105,745,866
Non-trainable params: 206
_________________________________________________________________
None


In [29]:
ae.output_shape

(None, 100, 100, 3)

# Callbacks

In [30]:
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [31]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-15, 
    patience=10, 
    verbose=1, 
    restore_best_weights=False
)

In [32]:
ms = ModelSaver(filepath=_restore)

In [33]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'experiments\\pokemonAE_Dense_reconst_ell\\csv_dir\\pokemonAE_Dense_reconst_ell.csv'

In [34]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [35]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

# Model Training

In [ ]:
ae.fit(
    x=train_generator,
    input_kw='images',
    steps_per_epoch=5,#int(5e3),
    epochs=5,#int(1e6), 
    verbose=1,
    #callbacks=[es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=train_generator,
    validation_steps=int(1e4)
)

Epoch 1/5
5/5 [==============================] - ETA: 0s - loss: 15731.1084

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=1000)
print(f'inception_score mean: {is_mean.compute()}, sigma: {is_sigma.compute()}')

## Frechet_inception_distance

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, train_generator.as_numpy_iterator(), 
                                       tolerance_threshold=1e-6, 
                                       max_iteration=1000, 
                                       batch_size=32)
print(f'frechet inception distance: {fis_score.compute()}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, train_generator.as_numpy_iterator(), 
                                              tolerance_threshold=1e-6, 
                                              max_iteration=1000, 
                                              batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae,  train_generator.as_numpy_iterator(), 
                                   tolerance_threshold=1e-6, 
                                   max_iteration=1000)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, train_generator.as_numpy_iterator(), tolerance_threshold=1e-6, max_iteration=1000)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
from training.generators.from_images.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, train_generator.as_numpy_iterator(), save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)

## with Randomness

In [ ]:
from training.generators.from_images.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, train_generator.as_numpy_iterator(), save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)

### Complete Randomness

In [ ]:
from training.generators.from_images.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [ ]:
from training.generators.from_images.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)